# Natural language processing (NLP) and probabilistic models

## Word-Processing Autocorref Algorithn compilation 

We use the autocorrect functionality everyday on our laptops, cell phones, computers etc. 

In this notebook, I aim to create a basic auto-correct tool and explore what goes on behind the scenes.

In [16]:
import nltk
import textdistance # lib designed ofr computing distance between words (strings)
# from nltk.corpus import words # English dictionary

from nltk.corpus import brown # Brown dictionary

import re
from collections import Counter

# Part A - WORDS

### Step 1:

1. Read the corpus (txt) file -> Download the corpus dictionary: Brown Dicionary
2. Change everything to lowercase
3. Return a list of words

In [21]:
# Download the Brown corpus if not already
# nltk.download("brown")

# Load Bown nglish words and create frequency dictionary - TOKENIZATION
vocab = brown.words()

# Create a frequency dictionary from the Brown corpus -  how many times each word appears in the Brown corpus.
vocab = Counter([w.lower() for w in vocab]) 

# Remove words that are not alphabetic or contain non-ASCII characters
# vocab = {word: freq for word, freq in vocab.items() if word.isalpha() and all(ord(c) < 128 for c in word)}

## Verify if words are on the dictionary (brown)
print(vocab['goin'])   # Should be > 0
print(vocab['going'])  # Should be much higher
print(vocab["water"]) 

4
399
445


Interprettion:

 "goin" is a real token in the Brown corpus — but it's informal/colloquial. 
 Since it exists, the code treats it as valid and doesn’t correct it.

### Autocorrect functions -  Levenshtein distance

In [ ]:
# Correction function V1 - Levenshtein distance
def get_closest_word(input_word, vocab=vocab, max_dist=2):
    input_word = input_word.lower()
    
    if input_word in vocab:
        return input_word
    
    # Generate possible corrections
    candidates = [word for word in vocab if textdistance.levenshtein(input_word, word) <= max_dist]
    
    if not candidates:
        return input_word  # No close match found

    # Return the most frequent close match
    return max(candidates, key=lambda w: vocab[w])

In [24]:
get_closest_word("goin")

'goin'

So, the code returns goin, as it is a toke, but we need to remove that

### V2 - Add a frequency threshold

#### Avoid returning low-frequency, informal words like "goin"

In [30]:
# Correction function V2 - Levenshtein distance - FREQUENCY THRESHOLD

def get_closest_word_v2(input_word, vocab=vocab, max_dist=2, min_freq=10):
    input_word = input_word.lower()
    
    # Only return it if it's common enough
    if input_word in vocab and vocab[input_word] >= min_freq:
        return input_word
    
    # Generate possible corrections
    candidates = [
        word for word in vocab 
        if textdistance.levenshtein(input_word, word) <= max_dist
        and vocab[word] >= min_freq  # Filter here too
    ]
    
    if not candidates:
        return input_word

    return max(candidates, key=lambda w: vocab[w])


In [ ]:
get_closest_word_v2("goin")

'in'

Intuition: 
* Function is working, but not giving the most semantically relevant result.
* FAUNCTION RETURNS the most (MAX) frequent word among all within max_dist = 2. Since "in" is a super common word in English, it's getting selected — even though "going" is a better correction for "goin".

### V3 - Score by distance and frequency

In [ ]:
def get_closest_word_v3(input_word, vocab=vocab, max_dist=2, min_freq=10):
    input_word = input_word.lower()
    
    # Only return it if it's common enough ( FREQUENCY THRESHOLD)
    if input_word in vocab and vocab[input_word] >= min_freq:
        return input_word

    candidates = []
    for word in vocab:
        if vocab[word] < min_freq:
            continue
        dist = textdistance.levenshtein(input_word, word)
        if dist <= max_dist:
            score = -dist + 0.001 * vocab[word]
            candidates.append((word, score))

    if not candidates:
        return input_word

    # Return word with best score
    return max(candidates, key=lambda x: x[1])[0]

print(get_closest_word_v3("goin"))





def get_closest_word_v4(input_word, vocab=vocab, max_dist=2, min_freq=10):
    input_word = input_word.lower()
    
    if input_word in vocab and vocab[input_word] >= min_freq:
        return input_word

    candidates = []
    for word in vocab:
        if vocab[word] < min_freq:
            continue

        dist = textdistance.levenshtein(input_word, word)

        if dist <= max_dist:
            # Penalize shorter words and large length differences
            len_diff_penalty = abs(len(input_word) - len(word)) * 0.5
            score = -dist - len_diff_penalty + 0.001 * vocab[word]
            candidates.append((word, score))

    if not candidates:
        return input_word

    return max(candidates, key=lambda x: x[1])[0]
print(get_closest_word_v4("goin"))




in
in


Now, you understood how difficult it is to build a good autocrref algorithn 

Let's build more complex score values:

* Adding a prefix constraint - first letter must have
* Penalizing edit distance (good)
* Boosting frequency using sqrt(freq) (smooths out the effect of very high-frequency words like "in")

In [ ]:

def get_candidates_v5(word, vocab=vocab, max_dist=2, top_n=5):
    word = word.lower()
    candidates = []

    for w in vocab:
        if w[0] != word[0]:  # Must match first letter
            continue

        dist = textdistance.levenshtein(word, w)
        if dist <= max_dist:
            freq = vocab[w]
            prefix_bonus = 1.5 if w.startswith(word[:3]) else 0
            score = -4 * dist + 0.6 * (freq ** 0.5) + prefix_bonus
            candidates.append((w, dist, freq, score))

    sorted_candidates = sorted(candidates, key=lambda x: x[3], reverse=True)

    print(f"\nSuggestions for '{word}':")
    for i, (w, d, f, s) in enumerate(sorted_candidates[:top_n], start=1):
        print(f"{i}. {w:<15} (distance={d}, freq={f}, score={s:.2f})")

    return [(w, d, f) for w, d, f, _ in sorted_candidates[:top_n]]

get_candidates_v5("goin")

get_candidates_v5("stp")



Suggestions for 'goin':
1. going           (distance=1, freq=399, score=9.48)
2. good            (distance=2, freq=806, score=9.03)
3. go              (distance=2, freq=626, score=7.01)
4. got             (distance=2, freq=482, score=5.17)
5. goin            (distance=0, freq=4, score=2.70)

Suggestions for 'stp':
1. she             (distance=2, freq=2860, score=24.09)
2. so              (distance=2, freq=1985, score=18.73)
3. see             (distance=2, freq=772, score=8.67)
4. say             (distance=2, freq=504, score=5.47)
5. set             (distance=2, freq=414, score=4.21)


[('she', 2, 2860),
 ('so', 2, 1985),
 ('see', 2, 772),
 ('say', 2, 504),
 ('set', 2, 414)]

### Does the logic works for other words like stop -  "stp"? 

It depends!! 

In [45]:
### Does the logic works for other words like stop -  "stp"?
get_candidates_v5("stp")
get_candidates_v5("maximn")


Suggestions for 'stp':
1. she             (distance=2, freq=2860, score=24.09)
2. so              (distance=2, freq=1985, score=18.73)
3. see             (distance=2, freq=772, score=8.67)
4. say             (distance=2, freq=504, score=5.47)
5. set             (distance=2, freq=414, score=4.21)

Suggestions for 'maximn':
1. maximum         (distance=2, freq=79, score=-1.17)
2. main            (distance=2, freq=119, score=-1.45)
3. maxim           (distance=1, freq=1, score=-1.90)
4. maximal         (distance=2, freq=3, score=-5.46)
5. maxine          (distance=2, freq=2, score=-5.65)


[('maximum', 2, 79),
 ('main', 2, 119),
 ('maxim', 1, 1),
 ('maximal', 2, 3),
 ('maxine', 2, 2)]

# Part B: Sentences

In [47]:
import re
import textdistance
from math import sqrt

VOWELS = set("aeiou")

def consonant_skeleton(w: str) -> str:
    w = w.lower()
    return "".join(ch for ch in w if ch.isalpha() and ch not in VOWELS)

def is_subsequence(small: str, large: str) -> bool:
    """Return True if all chars in `small` appear in order inside `large`."""
    it = iter(large)
    return all(c in it for c in small)

def score_candidate(candidate: str, original: str, dist: int, freq: int) -> float:
    # Base: strong penalty for distance, soft bonus for frequency
    score = -2.5 * dist + 0.1 * sqrt(freq)

    # Heuristic bonuses
    if consonant_skeleton(candidate) == consonant_skeleton(original):
        score += 12  # same consonant pattern (often missing vowel)
    if is_subsequence(original, candidate):
        score += 6   # original is a subsequence of candidate
    # Small prefix bonus
    if candidate.startswith(original[:2]):
        score += 2

    # Penalize very rare exact matches (e.g., slang/typos captured in corpus)
    if dist == 0 and freq < 10:
        score -= 4

    return score

def get_candidates_v2(word, max_dist=2, top_n=5):
    word = word.lower()
    candidates = []

    for w in vocab:  # vocab is your Counter from brown
        dist = textdistance.levenshtein(word, w)
        if dist <= max_dist:
            freq = vocab[w]
            s = score_candidate(w, word, dist, freq)
            candidates.append((w, dist, freq, s))

    candidates.sort(key=lambda x: x[3], reverse=True)

    # Debug print
    print(f"\nSuggestions for '{word}':")
    for i, (w, d, f, s) in enumerate(candidates[:top_n], start=1):
        print(f"{i}. {w} (distance={d}, freq={f}, score={s:.2f})")

    return [(w, d, f) for w, d, f, _ in candidates[:top_n]]

def clean_slang(word):
    """Fix known informal/slang before correction."""
    rules = {
        "goin": "going",
        "comin": "coming",
        "doin": "doing",
        "runnin": "running",
        "tryin": "trying",
        "wanna": "want to",
        "gonna": "going to"
    }
    return rules.get(word.lower(), word)

def correct_word(word, max_dist=2):
    word = clean_slang(word)
    suggestions = get_candidates_v2(word, max_dist=max_dist, top_n=1)
    return suggestions[0][0] if suggestions else word


In [48]:
import textdistance
print("going" in vocab)  
print(textdistance.levenshtein("goin", "going"))  # should return 2
print(vocab["going"])  # should return its frequency (you said ~399)


True
1
399


In [50]:

examples = [
    "I am goin to teh store",
    "Lets strt builidng autocrrect",
    "Recieve and adress should be corrected",
]

for sentences in examples:
    print("Original:", sentences)
    print("Corrected:", correct_sentence_v2(sentences))
    print("-" * 40)


Original: I am goin to teh store
['I', ' ', 'am', ' ', 'goin', ' ', 'to', ' ', 'teh', ' ', 'store']
Suggestions for 'I':

Suggestions for 'i':
1. , (distance=1, freq=58334, score=33.65)
Suggestions for 'am':

Suggestions for 'am':
1. am (distance=0, freq=237, score=21.54)
Suggestions for 'goin':

Suggestions for 'going':
1. going (distance=0, freq=399, score=22.00)
Suggestions for 'to':

Suggestions for 'to':
1. to (distance=0, freq=26158, score=36.17)
Suggestions for 'teh':

Suggestions for 'teh':
1. the (distance=2, freq=69971, score=33.45)
Suggestions for 'store':

Suggestions for 'store':
1. store (distance=0, freq=74, score=20.86)
Corrected: , am going to the store
----------------------------------------
Original: Lets strt builidng autocrrect
['Lets', ' ', 'strt', ' ', 'builidng', ' ', 'autocrrect']
Suggestions for 'Lets':

Suggestions for 'lets':
1. let's (distance=1, freq=69, score=18.33)
Suggestions for 'strt':

Suggestions for 'strt':
1. start (distance=1, freq=154, score=18